# Gathering Data

In [11]:
from functions import *
from scivol import *
import nibabel as nib
import ipyvolume as ipv

In [16]:
proj_root = parent_directory()
print(proj_root)
mri_input_filepath = os.path.join(proj_root, "media/sub-01/anat/sub-01_T2w.nii.gz")
output_folder = os.path.join(proj_root, "output/")
mask_seq_path = "/Users/joachimpfefferkorn/repos/neurovolume/output/slice_images_mask"
fmri = MRI_Anat(mri_input_filepath)

/Users/joachimpfefferkorn/repos/neurovolume


In [21]:
anat_img = nib.load(mri_input_filepath)
full_anat = create_volume(normalize_array(anat_img.get_fdata()))
brain_mask = build_bool_mask(mask_seq_path, full_anat)
brain_anat = create_masked_normalized_tensor(full_anat, brain_mask)
ext_anat = create_masked_normalized_tensor(full_anat, brain_mask, keep_when=False)
# don't love the inconsistent nesting of these volume creation functions 


bool masks built v1
creating masked normal tensor
creating masked normal tensor


# Quick Visual Check
Quick note on the following `ipv` functions:
They appear to be normalizing the geometry to be a perfect cube. I wonder if there is a way to apply the affine to this?

In [25]:
ipv.quickvolshow(full_anat)

Container(children=[VBox(children=(HBox(children=(Label(value='levels:'), FloatSlider(value=0.1, max=1.0, step…

In [26]:
ipv.quickvolshow(brain_anat)

Container(children=[VBox(children=(HBox(children=(Label(value='levels:'), FloatSlider(value=0.1, max=1.0, step…

In [27]:
ipv.quickvolshow(ext_anat)

Container(children=[VBox(children=(HBox(children=(Label(value='levels:'), FloatSlider(value=0.1, max=1.0, step…

# Building Scivol

In [28]:
affine = anat_img.affine
print(type(affine))

In [ ]:
skullstrip = Scivol("skullstrip", anat_img.affine, )